In [1]:
import os
import time
import numpy as np
import multiprocessing as mp
from multiprocessing import Process
from collections import Counter
from os.path import getsize

os.chdir("/Users/kobinae/KOBITemp/Dropbox/10_Venturus/MDC/201810_INF-0617/Ex01/INF0617_Ex01/gut/txt")
#print(os.getcwd())

filenames = []
for file in os.listdir("./"):
    if file.endswith(".txt"):
        filenames.append(file)
filenames_sorted = sorted(filenames, key=getsize)
##################################################################################
# Serial solution
##################################################################################
def processar_s(filename):
    f = open(filename, encoding='utf-8', errors='ignore')
    file_content = f.read()
    f.close()
    new_content = file_content.replace(' ', '').replace('\n','')
    return filename + ' ' + Counter(new_content).most_common(1)[0][0]
def serial(filenames,show_results):
    tmp_time = time.time()
    res = []
    for f in filenames:
        res.append(processar_s(f))
    if show_results:
        for r in res:
            print(r)
    return time.time()-tmp_time

##################################################################################
## Optimized paralel solution
##################################################################################
def processar_p(filename,q):
    f = open(filename, encoding='utf-8', errors='ignore')
    file_content = f.read()
    f.close()
    new_content = file_content.replace(' ', '').replace('\n','')
    q.put(filename + ' ' + Counter(new_content).most_common(1)[0][0])
def parallel_splited(filenames,nsplit,show_results):
    tmp_time = time.time()
    file_sets = np.array_split(np.array(filenames.copy()), nsplit)
    q = mp.Queue()
    for f_set in file_sets:
        for f in f_set:
            processes = []
            p = Process(target=processar_p, args=(f,q))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()
    if show_results:
        while not q.empty():
            print(q.get())
    return time.time() - tmp_time
##################################################################################
## Optimized paralel solution
##################################################################################
def read_p2(filename):
    f = open(filename, encoding='utf-8', errors='ignore')
    file_content = f.read()
    f.close()
    return file_content
def processar_p2(file_content,filename,q):
    new_content = file_content.replace(' ', '').replace('\n','')
    q.put(filename + ' ' + Counter(new_content).most_common(1)[0][0])

def parallel_splited2(filenames,nsplit,show_results):
    tmp_time = time.time()
    file_sets = np.array_split(np.array(filenames.copy()), nsplit)
    q = mp.Queue()
    for f_set in file_sets:
        contents=[]
        for i in range(0,len(f_set)-1):
            contents.append(read_p2(f_set[i]))
        for i in range(0,len(f_set)-1):
            processes = []
            p = Process(target=processar_p2, args=(contents[i],f_set[i],q))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()
    if show_results:
        while not q.empty():
            print(q.get())
    return time.time() - tmp_time


In [2]:
serial1_time = serial(filenames,False)
print('Serial execution took ' + str(serial1_time) + ' seconds')

Serial execution took 25.86820888519287 seconds


In [3]:
parallel_efficient_time = parallel_splited(filenames_sorted,4,False)
print('Paralle efficient execution took ' + str(parallel_efficient_time) + ' seconds')

Paralle efficient execution took 14.134817123413086 seconds


In [4]:
parallel_efficient_time = parallel_splited2(filenames_sorted,4,False)
print('Paralle efficient execution took ' + str(parallel_efficient_time) + ' seconds')

Paralle efficient execution took 14.480770111083984 seconds
